In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from math import sqrt
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [90]:
df_orig= pd.read_csv('df_base')

In [91]:
df= pd.read_csv('cleaned_df')
pd.DataFrame(df)
df.head()

,Unnamed: 0,id,budget,original_language,popularity,release_date,runtime,title,revenue
0,0,1,14000000,en,6.575393,2/20/15,93.0,Hot Tub Time Machine 2,12314651
1,1,2,40000000,en,8.248895,8/6/04,113.0,The Princess Diaries 2: Royal Engagement,95149435
2,2,3,3300000,en,64.299990,10/10/14,105.0,Whiplash,13092000
3,3,4,1200000,hi,3.174936,3/9/12,122.0,Kahaani,16000000
4,4,5,0,ko,1.148070,2/5/09,118.0,Marine Boy,3923970


In [92]:
df.head()

,Unnamed: 0,id,budget,original_language,popularity,release_date,runtime,title,revenue
0,0,1,14000000,en,6.575393,2/20/15,93.0,Hot Tub Time Machine 2,12314651
1,1,2,40000000,en,8.248895,8/6/04,113.0,The Princess Diaries 2: Royal Engagement,95149435
2,2,3,3300000,en,64.299990,10/10/14,105.0,Whiplash,13092000
3,3,4,1200000,hi,3.174936,3/9/12,122.0,Kahaani,16000000
4,4,5,0,ko,1.148070,2/5/09,118.0,Marine Boy,3923970


In [93]:
df_orig= df_orig.drop(columns=['poster_path', 'runtime', 'original_language','homepage', 'imdb_id', 'original_title', 'overview', 'status', 'Keywords', 'tagline', 'cast', 'crew', 'production_companies', 'production_countries', 'spoken_languages'])

In [94]:
df=df_orig

In [95]:
df.isnull().sum()

Unnamed: 0                  0
id                          0
belongs_to_collection    2396
budget                      0
genres                      7
popularity                  0
release_date                0
title                       0
revenue                     0
dtype: int64

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 9 columns):
Unnamed: 0               3000 non-null int64
id                       3000 non-null int64
belongs_to_collection    604 non-null object
budget                   3000 non-null int64
genres                   2993 non-null object
popularity               3000 non-null float64
release_date             3000 non-null object
title                    3000 non-null object
revenue                  3000 non-null int64
dtypes: float64(1), int64(4), object(4)
memory usage: 211.0+ KB


In [97]:
#need to get rid of nulls and replace with '0'
df= df.fillna(0)


In [98]:
#All films that beloing to a collection (book, franchise, etc.) assigned a 1
df.belongs_to_collection.loc[df['belongs_to_collection']!=0]=1

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [99]:
df['popularity'].describe()

count    3000.000000
mean        8.463274
std        12.104000
min         0.000001
25%         4.018053
50%         7.374861
75%        10.890983
max       294.337037
Name: popularity, dtype: float64

In [100]:
#create a feature that is gross profit pct_profit
df['gross_profit']= df['revenue']- df['budget']

In [101]:
df['gross_profit'].describe()

count    3.000000e+03
mean     4.419452e+07
std      1.123274e+08
min     -1.074474e+08
25%      1.795200e+04
50%      6.121122e+06
75%      4.143293e+07
max      1.316249e+09
Name: gross_profit, dtype: float64

In [102]:
#Creating a pct profit column
df['pct_profit']= (df['gross_profit']/df['revenue']*100).round(2)

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
Unnamed: 0               3000 non-null int64
id                       3000 non-null int64
belongs_to_collection    3000 non-null int64
budget                   3000 non-null int64
genres                   3000 non-null object
popularity               3000 non-null float64
release_date             3000 non-null object
title                    3000 non-null object
revenue                  3000 non-null int64
gross_profit             3000 non-null int64
pct_profit               3000 non-null float64
dtypes: float64(2), int64(6), object(3)
memory usage: 257.9+ KB


In [104]:
df['pct_profit'].describe()

count    3.000000e+03
mean    -1.978185e+05
std      4.921366e+06
min     -1.916666e+08
25%      1.466250e+01
50%      7.059000e+01
75%      1.000000e+02
max      1.000000e+02
Name: pct_profit, dtype: float64

In [116]:
#Will drop all movies with no budget listed
df = df[df.budget != 0]
df.describe()

,Unnamed: 0,id,belongs_to_collection,budget,popularity,revenue,gross_profit,pct_profit
count,2188.000000,2188.000000,2188.000000,2.188000e+03,2188.000000,2.188000e+03,2.188000e+03,2.188000e+03
mean,1507.420475,1508.420475,0.227605,3.089305e+07,9.700621,8.696605e+07,5.607299e+07,-2.712690e+05
std,863.873942,863.873942,0.419382,4.026785e+07,13.753541,1.554912e+08,1.285858e+08,5.761282e+06
min,0.000000,1.000000,0.000000,1.000000e+00,0.001880,1.000000e+00,-1.074474e+08,-1.916666e+08
25%,756.250000,757.250000,0.000000,5.150000e+06,5.398055,6.816828e+06,-1.606973e+06,-2.726250e+01
50%,1514.500000,1515.500000,0.000000,1.600000e+07,8.323165,2.891063e+07,1.034417e+07,5.009500e+01
75%,2245.500000,2246.500000,0.000000,4.000000e+07,11.657305,1.000342e+08,6.191157e+07,7.593750e+01
max,2999.000000,3000.000000,1.000000,3.800000e+08,294.337037,1.519558e+09,1.316249e+09,1.000000e+02


In [118]:
df.head()

,Unnamed: 0,id,belongs_to_collection,budget,genres,popularity,release_date,title,revenue,gross_profit,pct_profit
0,0,1,1,14000000,"[{'id': 35, 'name': 'Comedy'}]",6.575393,2/20/15,Hot Tub Time Machine 2,12314651,-1685349,-13.69
1,1,2,1,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",8.248895,8/6/04,The Princess Diaries 2: Royal Engagement,95149435,55149435,57.96
2,2,3,0,3300000,"[{'id': 18, 'name': 'Drama'}]",64.299990,10/10/14,Whiplash,13092000,9792000,74.79
3,3,4,0,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",3.174936,3/9/12,Kahaani,16000000,14800000,92.50
5,5,6,0,8000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",0.743274,8/6/87,Pinocchio and the Emperor of the Night,3261638,-4738362,-145.28
